# Edge Case: Federal CTC Expansion Can Reduce MA State Tax

This notebook demonstrates an edge case where federal CTC expansion reduces MA state tax when PolicyEngine's itemization logic uses `<=` instead of `<`.

In [ ]:
from policyengine_us import Simulation
from policyengine_us.model_api import *

## The Edge Case

PolicyEngine's itemization logic (when `branch_to_determine_itemization=true`) is:
```python
return tax_liability_if_itemizing < tax_liability_if_not_itemizing
```

This means when taxes are exactly equal, it doesn't itemize. But in some cases (like household 4428), the data shows itemization when taxes are equal, which could happen if:
1. There's rounding that makes them appear equal but they're not
2. The logic should be `<=` instead of `<`
3. There's some other factor at play

In [ ]:
# Create a scenario where we override the itemization logic
class ForceItemizeWhenEqual(Reform):
    def apply(self):
        self.modify_parameters(
            reform_class(
                name="force_itemize_when_equal",
                apply_to=["tax_unit_itemizes"],
                label="Force itemization when federal tax is equal",
            )
        )

@reform_class.add_reform(
    entity=TaxUnit,
    period=YEAR,
    variables=["tax_unit_itemizes"],
)
def tax_unit_itemizes(tax_unit, period, parameters):
    tax_liability_if_itemizing = tax_unit("tax_liability_if_itemizing", period)
    tax_liability_if_not_itemizing = tax_unit("tax_liability_if_not_itemizing", period)
    # Use <= instead of <
    return tax_liability_if_itemizing <= tax_liability_if_not_itemizing

# Create the situation
situation = {
    "people": {
        "parent1": {
            "age": {2026: 45},
            "employment_income": {2026: 17_237},
            "medical_out_of_pocket_expenses": {2026: 3_339},
        },
        "parent2": {
            "age": {2026: 43},
        },
        "child1": {"age": {2026: 13}},
        "child2": {"age": {2026: 12}},
        "child3": {"age": {2026: 6}},
        "child4": {"age": {2026: 4}},
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent1", "parent2", "child1", "child2", "child3", "child4"],
        }
    },
    "households": {
        "household": {
            "members": ["parent1", "parent2", "child1", "child2", "child3", "child4"],
            "state_code": {2026: "MA"},
        }
    },
}

## Demonstrate the Effect

If the issue is indeed about how PolicyEngine handles the equal case, then this demonstrates how federal CTC expansion could reduce MA state tax.

In [ ]:
print("This demonstrates that IF PolicyEngine used <= instead of < for itemization,")
print("then federal CTC expansion could reduce MA state tax through this mechanism:")
print()
print("1. CTC expansion makes federal tax equal whether itemizing or not")
print("2. With <=, PolicyEngine would itemize when indifferent")
print("3. MA allows itemizers to claim medical expense deduction")
print("4. This reduces MA state tax")
print()
print("The actual behavior in household 4428 suggests this might be happening,")
print("possibly due to rounding or other factors.")